# Experiment with the dvd rental database

In [8]:
%run connection.ipynb

Connecting with connection string : postgresql://postgres:letmein@db:5432/dvdrental
 * postgresql://postgres:***@db:5432/dvdrental
1 rows affected.
 * postgresql://postgres:***@db:5432/dvdrental
1 rows affected.
 * postgresql://postgres:***@db:5432/dvdrental
15 rows affected.


### How much each customer paid for their first rental?

In [9]:
%%sql

WITH base_table AS (
    SELECT
        r.customer_id,
        MIN(r.rental_id) AS first_rental_id,
        (
            SELECT r2.rental_date FROM rental r2
            WHERE r2.rental_id = MIN(r.rental_id)
        )::date AS first_rental_date
    FROM rental r
    GROUP BY 1
    ORDER BY 1
)

SELECT
    bt.*,
    p.amount
FROM base_table bt
    LEFT JOIN payment p ON p.customer_id = bt.customer_id
        AND p.rental_id = bt.first_rental_id

GROUP BY 1,2,3,4
ORDER BY 1

 * postgresql://postgres:***@db:5432/dvdrental
599 rows affected.


,customer_id,first_rental_id,first_rental_date,amount
0,1,76,2005-05-25,NaN
1,2,320,2005-05-27,NaN
2,3,435,2005-05-27,NaN
3,4,1297,2005-06-15,4.99
4,5,731,2005-05-29,NaN
...,...,...,...,...
594,595,613,2005-05-28,NaN
595,596,303,2005-05-26,NaN
596,597,34,2005-05-25,NaN
597,598,3005,2005-06-20,2.99


### What are the top ten most paid customers?

In [10]:
%%sql

SELECT
    SUM(p.amount) AS total_payment_amount,
    r.customer_id,
    (c.first_name || ' ' || c.last_name) as customer

FROM payment p
    INNER JOIN rental r ON r.rental_id = p.rental_id
    INNER JOIN customer c ON c.customer_id = r.customer_id

GROUP BY 2,3
ORDER BY 1 DESC
LIMIT 10


 * postgresql://postgres:***@db:5432/dvdrental
10 rows affected.


,total_payment_amount,customer_id,customer
0,211.55,148,Eleanor Hunt
1,208.58,526,Karl Seal
2,194.61,178,Marion Snyder
3,191.62,137,Rhonda Kennedy
4,189.60,144,Clara Shaw
5,183.63,459,Tommy Collazo
6,167.67,181,Ana Bradley
7,167.62,410,Curtis Irby
8,166.61,236,Marcia Dean
9,162.67,403,Mike Way


### How much each customer paid in their first payment and which film they rent?

In [11]:
%%sql

WITH base_table AS (
    SELECT p.*
    FROM payment p
    WHERE p.payment_id = (
        SELECT MIN(p2.payment_id) FROM payment p2
        WHERE p2.customer_id = p.customer_id
    )
    ORDER BY 2,6
)

SELECT
    bt.customer_id,
    bt.amount,
    f.title as film_title
FROM base_table bt
    INNER JOIN rental r ON r.rental_id = bt.rental_id
    INNER JOIN inventory i ON i.inventory_id = r.inventory_id
    INNER JOIN film f ON f.film_id = i.film_id

ORDER BY 1

 * postgresql://postgres:***@db:5432/dvdrental
599 rows affected.


,customer_id,amount,film_title
0,1,5.99,Musketeers Wait
1,2,2.99,Blackout Private
2,3,8.99,Strangers Graffiti
3,4,4.99,Dogma Family
4,5,3.99,Reef Salute
...,...,...,...
594,595,2.99,Lost Bird
595,596,1.99,Coast Rainbow
596,597,0.99,Moon Bunch
597,598,2.99,Beethoven Exorcist


### What films each customer rent?

In [12]:
%%sql

SELECT
    r.customer_id,
    ARRAY_AGG(film.title) AS all_retnted_films,
    array_length(ARRAY_AGG(film.title), 1) AS total_films_rent

FROM payment p
    INNER JOIN rental r ON r.rental_id = p.rental_id

    INNER JOIN (
        SELECT i.film_id, i.inventory_id FROM inventory i
    ) AS inventory ON inventory.inventory_id = r.inventory_id
    
    INNER JOIN (
        SELECT f.title, f.film_id FROM film f
    ) AS film ON film.film_id = inventory.film_id

GROUP BY 1
ORDER BY 3 DESC

 * postgresql://postgres:***@db:5432/dvdrental
599 rows affected.


,customer_id,all_retnted_films,total_films_rent
0,148,"[Spirited Casualties, Outlaw Hanky, Boulevard ...",45
1,526,"[Sting Personal, Sinners Atlantis, Betrayed Re...",42
2,144,"[Network Peak, Harry Idaho, Patriot Roman, Ran...",40
3,236,"[Gathering Calendar, Chamber Italian, Madre Ga...",39
4,75,"[Sun Confessions, Fatal Haunted, Splendor Patt...",39
...,...,...,...
594,310,"[Harold French, Magnolia Forrester, Airplane S...",13
595,248,"[Armageddon Lost, Illusion Amelie, Hoosiers Bi...",13
596,110,"[Paths Control, Comforts Rush, Brotherhood Bla...",12
597,281,"[Packer Madigan, Dynamite Tarzan, Wisdom Worke...",10
